In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj, transform
from tqdm import tqdm
import fiona



# Read the dataset with longitude and latitude of events
data = pd.read_csv('..\\1.Initilization\\ACLED.csv',sep="\t")  # Replace 'events.csv' with the path to your dataset

# Read the areas GeoJSON file
areas_gdf = gpd.read_file('..\\1.Initilization\\output.geojson')

data.describe()

# extract month and year from the event_date column
data['year'] = pd.DatetimeIndex(data['event_date']).year
data['month'] = pd.DatetimeIndex(data['event_date']).month

# create a new column combine year and month
data['year_month'] = data['year'].astype(str) + '-' + data['month'].astype(str)

# convert the year_month column to datetime
data['year_month'] = pd.to_datetime(data['year_month'])

# convert areas_gdf from column to datetime, Apr-17 to 2017-04-01
areas_gdf['from'] = pd.to_datetime(areas_gdf['from'])
# Convert the DataFrame into a GeoDataFrame with Points
geometry = [Point(xy) for xy in zip(data['longitude'], data['latitude'])]
data_gdf = gpd.GeoDataFrame(data, geometry=geometry)

# transform the coordinate system
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
x1,y1 = data_gdf['geometry'].x, data_gdf['geometry'].y
x2,y2 = transform(inProj,outProj,x1,y1)
data_gdf['geometry'] = [Point(xy) for xy in zip(x2, y2)]

# unique date
date = areas_gdf['from'].unique()

C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_21088\1646597311.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  areas_gdf['from'] = pd.to_datetime(areas_gdf['from'])
c:\Users\WeilunShi\.virtualenvs\Food_Crisis_and_Insecurity-8fC6jjVv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\WeilunShi\.virtualenvs\Food_Crisis_and_Insecurity-8fC6jjVv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the chang

In [2]:


results = []

# loop through the years
for dat in date:
    # Filter events and areas by the current date
    areas_current_year = areas_gdf[areas_gdf['from'] == dat]  # Assuming the areas are the same for all years

    # filter events within three months 
    events_current_year = data_gdf[(data_gdf['year_month'] >= dat - pd.DateOffset(months=3)) & (data_gdf['year_month'] <= dat)]

    # Perform the spatial join
    data_gdf_with_areas = gpd.sjoin(events_current_year, areas_current_year, how='right', op='within')

    # Group by area and type of events, and count the number of events
    data_gdf_with_areas_event_counts = data_gdf_with_areas.groupby(['title','event_type']).size().reset_index(name='counts')

    # Group by area, count the number of fatalities
    data_gdf_with_areas_fatalities = data_gdf_with_areas.groupby(['title'])['fatalities'].sum().reset_index(name='fatalities')

    # Merge the two dataframes
    data_gdf_with_areas_event_counts = data_gdf_with_areas_event_counts.merge(data_gdf_with_areas_fatalities, on='title')

    # reshape the data
    data_gdf_with_areas_event_counts = data_gdf_with_areas_event_counts.pivot(index='title', columns='event_type', values=['counts','fatalities']).reset_index()

    # combine the headers
    data_gdf_with_areas_event_counts.columns = ['_'.join(col).strip() for col in data_gdf_with_areas_event_counts.columns.values]

    # rename the columns
    data_gdf_with_areas_event_counts.rename(columns={'title_':'title','counts_Battles':'battles','counts_Explosions/Remote violence':'explosions','counts_Violence against civilians':'violence','fatalities_Battles':'fatalities_battles','fatalities_Explosions/Remote violence':'fatalities_explosions','fatalities_Violence against civilians':'fatalities_violence'}, inplace=True)

    # fill the missing values with 0
    data_gdf_with_areas_event_counts.fillna(0, inplace=True)

    # merge with the areas
    areas_gdf_with_events = areas_gdf.merge(data_gdf_with_areas_event_counts, on='title', how='left')

    # append the results

    results.append(areas_gdf_with_events)

# concatenate the results
results = pd.concat(results)


# drop the columns
results = results.drop(['period','to','color','phase3_worse_population','phase3_worse_percentage','phase1_color','phase2_color','phase3_color','phase4_color','phase5_color'], axis=1)


# rename 'from' to 'date'
results.rename(columns={'from':'date'}, inplace=True)

# fill the missing values with 0
results.fillna(0, inplace=True)
results['geometry'] = results['geometry'].buffer(0)

c:\Users\WeilunShi\.virtualenvs\Food_Crisis_and_Insecurity-8fC6jjVv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_21088\1625481886.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3857

  data_gdf_with_areas = gpd.sjoin(events_current_year, areas_current_year, how='right', op='within')
c:\Users\WeilunShi\.virtualenvs\Food_Crisis_and_Insecurity-8fC6jjVv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, resul

In [3]:

# Create a new column that combines 'area' and 'date' as a tuple
results['agg_column'] = list(zip(results['title'], results['date']))

# Use the dissolve method to aggregate the GeoDataFrame
aggregated_gdf = results.dissolve(by='agg_column', aggfunc={'battles':'sum', 'explosions':'sum', 'violence':'sum', 'fatalities_battles':'sum', 'fatalities_explosions':'sum', 'fatalities_violence':'sum', 'id':'first', 'estimated_population':'first', 'anl_id':'first','overall_phase':'first', 'country':'first', 'year':'first', 'condition':'first', 'phase1_population':'first',
'phase1_percent':'first', 'phase2_population':'first', 'phase2_percent':'first','phase3_population':'first', 'phase3_percent':'first', 'phase4_population':'first','phase4_percent':'first', 'phase5_population':'first', 'phase5_percent':'first','phase3_worse_percentage_manual':'first', 'phase3_plus_phase4':'first',
'phase2_worse_percentage_manual':'first', 'phase2_plus_phase3':'first','phase2_plus_phase3_plus_phase4':'first'})

# Reset the index to have a clean GeoDataFrame
aggregated_gdf.reset_index(inplace=True)

# Separate the 'area' and 'date' columns from the tuple in 'agg_column'
aggregated_gdf[['title', 'date']] = gpd.GeoDataFrame(aggregated_gdf['agg_column'].tolist(), index=aggregated_gdf.index)

# Drop the 'agg_column' as it is not needed anymore
aggregated_gdf.drop(columns='agg_column', inplace=True)

aggregated_gdf.head()


,geometry,battles,explosions,violence,fatalities_battles,fatalities_explosions,fatalities_violence,id,estimated_population,anl_id,...,phase4_percent,phase5_population,phase5_percent,phase3_worse_percentage_manual,phase3_plus_phase4,phase2_worse_percentage_manual,phase2_plus_phase3,phase2_plus_phase3_plus_phase4,title,date
0,"POLYGON ((1810577.311 -2977417.202, 1818626.29...",0.0,0.0,0.0,0.0,0.0,0.0,26893518,96015,26893489,...,0.00,0.0,0.0,0.15,0.15,0.45,0.45,0.45,//Kharas,2022-09-01
1,"MULTIPOLYGON (((5202284.611 -2893069.833, 5201...",0.0,0.0,0.0,0.0,0.0,0.0,12151688,37182,12151682,...,0.09,0.0,0.0,0.00,0.44,0.00,0.75,0.84,4 commune de taolanaro,2017-03-01
2,"MULTIPOLYGON (((5202284.611 -2893069.833, 5201...",0.0,0.0,0.0,0.0,0.0,0.0,12151845,37182,12151843,...,0.18,0.0,0.0,0.65,0.65,0.90,0.72,0.90,4 commune de taolanaro,2017-08-01
3,"MULTIPOLYGON (((5202284.611 -2893069.833, 5201...",0.0,0.0,0.0,0.0,0.0,0.0,12632339,38223,12632331,...,0.06,0.0,0.0,0.18,0.18,0.68,0.62,0.68,5 communes de Taolagnaro,2018-03-01
4,"MULTIPOLYGON (((5202284.611 -2893069.833, 5201...",0.0,0.0,0.0,0.0,0.0,0.0,12940714,38223,12940680,...,0.06,0.0,0.0,0.19,0.19,0.70,0.64,0.70,5 communes de Taolagnaro,2018-08-01


In [4]:
aggregated_gdf.describe()

,battles,explosions,violence,fatalities_battles,fatalities_explosions,fatalities_violence,estimated_population,overall_phase,year,phase1_population,...,phase4_population,phase4_percent,phase5_population,phase5_percent,phase3_worse_percentage_manual,phase3_plus_phase4,phase2_worse_percentage_manual,phase2_plus_phase3,phase2_plus_phase3_plus_phase4,date
count,9381.000000,9381.000000,9381.000000,9381.000000,9381.000000,9381.000000,9.381000e+03,9381.000000,9381.000000,9.381000e+03,...,9.381000e+03,9381.000000,9381.000000,9381.000000,9381.000000,9381.000000,9381.000000,9381.000000,9381.000000,9381
mean,29.305831,13.133781,9.265963,145.893828,130.683722,136.177167,3.024279e+05,2.607398,2020.085066,1.209591e+05,...,1.723895e+04,0.065538,151.086025,0.001009,0.267462,0.267481,0.568008,0.507786,0.569266,2020-07-10 17:06:00.345378816
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2017.000000,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2017-01-01 00:00:00
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.725400e+04,2.000000,2019.000000,1.428100e+04,...,0.000000e+00,0.000000,0.000000,0.000000,0.110000,0.120000,0.400000,0.400000,0.400000,2019-05-01 00:00:00
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.408030e+05,3.000000,2020.000000,4.673500e+04,...,4.000000e+03,0.050000,0.000000,0.000000,0.220000,0.220000,0.600000,0.550000,0.600000,2020-08-01 00:00:00
75%,7.000000,1.000000,6.000000,24.000000,0.000000,22.000000,3.409520e+05,3.000000,2022.000000,1.263380e+05,...,1.655200e+04,0.100000,0.000000,0.000000,0.400000,0.400000,0.750000,0.650000,0.750000,2022-03-01 00:00:00
max,1450.000000,828.000000,401.000000,10802.000000,10802.000000,10802.000000,3.170700e+07,9.000000,2023.000000,7.501845e+06,...,5.619500e+06,0.620000,143804.000000,0.300000,0.950000,0.900000,1.020000,1.000000,1.020000,2023-02-01 00:00:00
std,126.203916,64.340182,30.665635,708.477537,706.762327,699.432910,5.954257e+05,1.029162,1.678604,2.717019e+05,...,7.012167e+04,0.076395,2581.749326,0.009352,0.196733,0.193915,0.257365,0.209447,0.253785,NaN


In [5]:
# save the results
aggregated_gdf.to_file('output.geojson', driver='GeoJSON')